In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/pollution.csv')

In [4]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'date'], infer_datetime_format=True)

In [6]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0,2010-01-02 00:00:00
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0,2010-01-02 01:00:00
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0,2010-01-02 02:00:00
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0,2010-01-02 03:00:00
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [7]:
df.columns

Index(['date', 'pollution', 'dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain',
       'NE', 'NW', 'SE', 'cv', 'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(35040, 13)

In [10]:
test_df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
35040,2014-01-01 00:00:00,24.0,-20,7.0,1014.0,143.48,0,0,0,1,0,0,2014-01-01 00:00:00
35041,2014-01-01 01:00:00,53.0,-20,7.0,1013.0,147.50,0,0,0,1,0,0,2014-01-01 01:00:00
35042,2014-01-01 02:00:00,65.0,-20,6.0,1013.0,151.52,0,0,0,1,0,0,2014-01-01 02:00:00
35043,2014-01-01 03:00:00,70.0,-20,6.0,1013.0,153.31,0,0,0,1,0,0,2014-01-01 03:00:00
35044,2014-01-01 04:00:00,79.0,-18,3.0,1012.0,0.89,0,0,0,0,0,1,2014-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [11]:
train_ds = PandasDataset(
    train_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

In [12]:
test_ds = PandasDataset(
    test_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

#### training

In [13]:
K = 24
n_hours = 24*3

In [ ]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=200,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-08,)

estimator = DeepAREstimator(
    prediction_length=K,
    freq="H",
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    batch_size=128,
    trainer=trainer,
    dropout_rate=0.3,
)

model = estimator.train(train_ds)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:23<00:00,  3.84s/it, epoch=76/200, avg_epoch_loss=4.04]


 72%|█████████████████████████████████████████████████████████████████████████████████████████████████▉                                      | 72/100 [04:32<01:45,  3.75s/it, epoch=105/200, avg_epoch_loss=3.99]

#### predicted y

In [ ]:
quantiles = ['0.025', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.975']
q50 = 5

In [ ]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].samples
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

In [ ]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

In [ ]:
test_ds_predict_q50_supervised[0:5]

#### actual y

In [ ]:
from helper import series_to_supervised, stage_series_to_supervised

In [ ]:
test_df_supervised = series_to_supervised(test_df[['pollution']], n_hours, K)

In [ ]:
test_df_supervised.iloc[:, -K:].shape

In [ ]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [ ]:
test_df_supervised_y

#### performance on test set

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))